In [1]:
#!pip install georasters
#!pip install geoindex
#!pip install spark
#!pip install pyspark

In [3]:
import pandas as pd
import georasters as gr
import matplotlib.pyplot as plt
from geoindex import GeoGridIndex, GeoPoint
import os
from tqdm import tqdm, tqdm_pandas

In [4]:
pd.set_option('display.max_columns', 100)

## Load Water Data

In [5]:
os.getcwd()

'/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access/Population'

In [6]:
water_data = pd.read_csv("/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access//data/Water_Point_Data_Exchange_Complete_Dataset.csv")

/Users/alexjamar/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,5,7,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
water_data.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,NaN,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,SNV,Arusha,Longido,wpdx-00000001,01/05/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.83257,36.3090,"(-2.83257, 36.309)",1,NaN,NaN,NaN
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,SNV,Arusha,Longido,wpdx-00000002,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84781,36.5284,"(-2.84781, 36.5284)",1,NaN,NaN,NaN
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000003,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84796,36.5288,"(-2.84796, 36.5288)",1,NaN,NaN,NaN
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000004,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.85318,36.5276,"(-2.85318, 36.5276)",1,NaN,NaN,NaN
4,5,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,El Porvenier,Matagalpa,Terrabona,wpdx-00000005,01/01/2011 12:00:00 AM +0000,NI,NaN,http://www.dropbox.com/s/0f70mln5roxhy29/ProjT...,https://www.waterpointdata.org/dataset/resourc...,NaN,"#data_lnk, #lon_deg, #lat_deg, #country_id",04/08/2016 08:35:49 AM +0000,04/08/2016 08:35:49 AM +0000,12.74070,-86.0121,"(12.7407, -86.0121)",1,NaN,NaN,NaN


In [8]:
kenya_water = water_data[water_data['#country_name'] == 'Kenya']

In [9]:
kenya_water.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
39068,59708,Kenya,NaN,Mechanized Borehole-Grid,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00059708,05/30/2013 12:00:00 AM +0000,KE,Chepkulul Borehole,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 09:24:31 PM +0000,10/26/2015 09:24:31 PM +0000,0.09260,36.1744,"(0.0926, 36.1744)",1,NaN,NaN,NaN
39075,59715,Kenya,NaN,Mechanized Borehole-Grid,yes,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00059715,09/09/2011 12:00:00 AM +0000,KE,Marana Phase 11,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 09:24:31 PM +0000,10/26/2015 09:24:31 PM +0000,0.12480,36.2192,"(0.1248, 36.2192)",1,NaN,NaN,NaN
39078,59718,Kenya,NaN,Mechanized Borehole-Grid,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00059718,04/18/2012 12:00:00 AM +0000,KE,Tuiyotich community water project,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 09:24:31 PM +0000,10/26/2015 09:24:31 PM +0000,0.04610,36.1409,"(0.0461, 36.1409)",1,NaN,NaN,NaN
39079,59719,Kenya,NaN,Mechanized Borehole-Grid,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00059719,05/20/2010 12:00:00 AM +0000,KE,Wiyumirerrie water project,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 09:24:31 PM +0000,10/26/2015 09:24:31 PM +0000,0.09500,36.2155,"(0.095, 36.2155)",1,NaN,NaN,NaN
39108,59748,Kenya,NaN,Mechanized Borehole-Grid,yes,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00059748,07/09/2012 12:00:00 AM +0000,KE,Machine Community project,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 09:24:31 PM +0000,10/26/2015 09:24:31 PM +0000,0.08482,36.1479,"(0.08482, 36.1479)",1,NaN,NaN,NaN


## Radius-based Search

In [10]:
kenya_pop = pd.read_csv("Kenya_pop_data.csv")

In [ ]:
#Create Geo Index of Kenya Population Data
geo_index = GeoGridIndex()
for index, row in kenya_pop.iterrows():
    geo_index.add_point(GeoPoint(row['y'], row['x'], ref=row['value']))

In [ ]:
#Create Geo Index of Kenya Water Point Data
geo_index_water = GeoGridIndex()
for index, row in kenya_water.iterrows():
    try:
        geo_index_water.add_point(GeoPoint(row['#lat_deg'], row['#lon_deg'], ref=row['#status_id']))
    except:
        print("Row ID:", row['Row ID'])
        print("Lat:", row['#lat_deg'])
        print("Long:", row['#lon_deg'])

In [ ]:
#Calculate population within x distance of water point
def calculate_population_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_population = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_population += point.ref
    except:
        print("Invalid data - Record skipped")
    #print("Total population within", km, "kilometers:", int(total_population))
    return total_population

In [ ]:
#Calculate # of water points within x distance of water point
def calculate_water_points_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_water_points = 0
    total_functioning_water_points = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_water_points += 1
            if point.ref == 'yes':
                total_functioning_water_points += 1
    except:
        print("Invalid data - Record skipped")
        
    #Adjust for self
    total_water_points -= 1
    if row['#status_id'] == 'yes':
        total_functioning_water_points -= 1
    return (total_water_points, total_functioning_water_points)

In [ ]:
#Add 1 km radius population data to each well
tqdm.pandas()
kenya_water['1 km population'] = kenya_water.progress_apply(lambda row: calculate_population_within_x_km(row, geo_index, 1),
                                                  axis = 1)

In [ ]:
#Add 1 km radius water point data to each water point
tqdm.pandas()
kenya_water['1 km water points (total, functioning)'] = kenya_water.progress_apply(lambda row: calculate_water_points_within_x_km(row, geo_index_water, 1),
                                                  axis = 1)

In [ ]:
kenya_water.head(2)

In [ ]:
def clean_water_data(df):
    df['1 km population'] = df['1 km population'].apply(lambda x: int(round(x,0)))
    df['1 km total water points'] = df['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[0][1:])
    df['1 km functioning water points'] = df['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[1][:-1])
    df = df[['Row ID', '#country_name', '#status_id', '#lat_deg', '#lon_deg', '1 km population',
                        '1 km total water points', '1 km functioning water points']]
    
    return df

In [ ]:
kenya_abridged = clean_water_data(kenya_water)

In [ ]:
kenya_abridged.head()

In [20]:
liberia_abridged.to_csv("Abridged_Liberia_Water_and_Population_Data.csv", index = False)